# Kaggle - Web Traffic Time Series Forecasting

## Model 03 - Weekday v3 - Median 49

by Louis Yang

Find the median visits in each of page at each weekday (Monday, Tuesday, ..., Sunday) for the last 49 days of visit. Predict the future visit using these medians in each weekday.

This script doesn't depend on any other code in this project.

In [1]:
model_name = 'model_03_weekday_3_median_49'

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

## Input

In [3]:
print('%%% Reading data train_1.csv ... ', end = '', flush = True)
train = pd.read_csv("../input/train_1.csv")
print('done!')

In [4]:
train

In [5]:
train.info()

## Model

In [6]:
# Take only the last 49 days of data and Flattend the 'date' dimension for each 'Page'
train_flattened = pd.melt(train[list(train.columns[-49:])+['Page']], 
                                id_vars='Page', var_name='date', 
                                value_name='Visits')
train_flattened

In [7]:
train_flattened['date'] = train_flattened['date'].astype('datetime64[ns]')
train_flattened['weekday'] = train_flattened.date.dt.dayofweek
train_flattened

In [8]:
train_flattened.groupby(['Page','weekday']).median()  # compute the median of visits of each page in each weekday

Compute the median of visits of each page in each weekday

In [9]:
train_page_per_dow = train_flattened.groupby(['Page','weekday']).median().reset_index()
train_page_per_dow  # no date columns remained

## Output

In [10]:
print('%%% Reading data key_1.csv ...', end = '', flush = True)
test = pd.read_csv("../input/key_1.csv")
print('done!')

In [11]:
test

In [12]:
test['date'] = test.Page.apply(lambda a: a[-10:])  # take the last 10 characters from 'Page' as date
test['Page'] = test.Page.apply(lambda a: a[:-11])  # remove the last 10 caharacters from 'Page'
test['date'] = test['date'].astype('datetime64[ns]')  # convert 'date' string to numpy datetime format
test['weekday'] = test.date.dt.dayofweek  # find the day of week using the 'date' column
test

Add and fill the 'Visits' column from "train_page_per_dow" to "test" with the same weekday and page

In [13]:
test = test.merge(train_page_per_dow, how='left')
test.loc[test.Visits.isnull(), 'Visits'] = 0.0  # Replace NaN with 0.0
test

In [14]:
print('%%% Writing result for ' + model_name + ' ...', end = '', flush = True)
# Write only the 'Id' and 'Visits' to the result file
test[['Id','Visits']].to_csv('submit_1_' + model_name
                             + '.csv', index = False)
print('done!')